# Tutorial 1 - RNN Sequence classifier

In this notebook, we will predict the winner of a basketball game based on the scores observed in the first 3 quarters of the game. Each column represents the beginning of a minute during the game. (There are 12 minutes in each quarter. There are  3 quarters in the data, so we have 36 columns as input variables in chronological order.) The values captured in each column represent the score difference observed at that minute (home score minus away score)<br><br>

The last column `W` represents whether the home team (1) or the away team (0) won the game. This is the target variable. <br><br>
**Our unit of analysis is a single game.**

In [1]:
# Common imports
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

## Get the data

In [2]:
data = pd.read_csv("basketball.csv")

In [3]:
data.shape

(1230, 37)

In [4]:
data.head()

,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10,...,M28,M29,M30,M31,M32,M33,M34,M35,M36,W
0,-2,-1,1,1,-1,-3,-1,0,3,6,...,9,11,10,7,7,4,6,2,1,1
1,0,2,7,6,10,8,8,6,0,6,...,7,11,11,14,14,15,13,13,13,0
2,0,-2,2,0,5,4,5,3,5,3,...,9,11,13,13,12,17,15,15,12,1
3,0,2,0,3,4,3,5,4,3,1,...,10,6,7,7,8,8,8,8,8,1
4,0,-2,-2,0,3,-2,-7,-5,-7,-4,...,10,10,15,13,11,11,11,13,10,1


In [5]:
y = data['W']
x = data.drop('W', axis=1)

## Split the data

In [6]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3)

# Can even shuffle the data, for keras order or games does not matter . but coloumns sequence are important

## Data Transformation

In [7]:
#Target variables need to be an array with integer type
train_y = np.array(train_y)
test_y = np.array(test_y)

train_y = train_y.astype(np.int32)
test_y = test_y.astype(np.int32)



In [8]:
#Check the first 10 values of the train_y data set
train_y[0:10]

array([1, 1, 0, 0, 1, 0, 1, 1, 0, 1])

In [9]:
#Convert input variables to a 2-D array with float data type
train_x= np.array(train_x)
test_x= np.array(test_x)

train_x = train_x.astype(np.float32)
test_x = test_x.astype(np.float32)

In [10]:
train_x

array([[  0.,  -4.,   1., ...,   6.,   4.,   6.],
       [  3.,   7.,   8., ...,  22.,  20.,  20.],
       [  0.,   2.,   5., ...,  -8.,  -9., -14.],
       ...,
       [ -4.,  -4.,  -3., ..., -13., -12., -13.],
       [ -1.,  -1.,   2., ...,   9.,   9.,  10.],
       [  1.,   5.,   9., ...,  23.,  23.,  21.]], dtype=float32)

In [11]:
#Keras expects a different input format:
#Data needs to have 3 dimensions

train_x = np.reshape(train_x, (train_x.shape[0], train_x.shape[1], 1))
test_x = np.reshape(test_x, (test_x.shape[0], test_x.shape[1], 1))



In [12]:
train_x.shape, train_y.shape

((861, 36, 1), (861,))

In [13]:
train_x

array([[[  0.],
        [ -4.],
        [  1.],
        ...,
        [  6.],
        [  4.],
        [  6.]],

       [[  3.],
        [  7.],
        [  8.],
        ...,
        [ 22.],
        [ 20.],
        [ 20.]],

       [[  0.],
        [  2.],
        [  5.],
        ...,
        [ -8.],
        [ -9.],
        [-14.]],

       ...,

       [[ -4.],
        [ -4.],
        [ -3.],
        ...,
        [-13.],
        [-12.],
        [-13.]],

       [[ -1.],
        [ -1.],
        [  2.],
        ...,
        [  9.],
        [  9.],
        [ 10.]],

       [[  1.],
        [  5.],
        [  9.],
        ...,
        [ 23.],
        [ 23.],
        [ 21.]]], dtype=float32)

# Baseline Accuracy

In [14]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(train_x, train_y)

DummyClassifier(strategy='most_frequent')

In [15]:
from sklearn.metrics import accuracy_score

In [16]:
#Baseline Train Accuracy
dummy_train_pred = dummy_clf.predict(train_x)

baseline_train_acc = accuracy_score(train_y, dummy_train_pred)

print('Baseline Train Accuracy: {}' .format(baseline_train_acc))

Baseline Train Accuracy: 0.5586527293844367


In [17]:
#Baseline Test Accuracy
dummy_test_pred = dummy_clf.predict(test_x)

baseline_test_acc = accuracy_score(test_y, dummy_test_pred)

print('Baseline Test Accuracy: {}' .format(baseline_test_acc))

Baseline Test Accuracy: 0.5365853658536586


# A normal (cross-sectional) NN

This model assumes that the data is NOT a time-series data set. It treats the data as cross-sectional and the columns being independent of each other.

In [18]:
model = keras.models.Sequential([
    
    keras.layers.Flatten(input_shape=[36, 1]),
    keras.layers.Dense(36, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
    
])

C:\Users\mural\anaconda3\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [19]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

# If multiclass, use "sparse_categorical_crossentropy" as the loss function
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])


history = model.fit(train_x, train_y, epochs=50,
                    validation_data=(test_x, test_y))

Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.6977 - loss: 2.3529 - val_accuracy: 0.7507 - val_loss: 0.8561
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8101 - loss: 0.5433 - val_accuracy: 0.7371 - val_loss: 0.7584
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7985 - loss: 0.4935 - val_accuracy: 0.7453 - val_loss: 0.6069
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8228 - loss: 0.4465 - val_accuracy: 0.7534 - val_loss: 0.5427
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8241 - loss: 0.3962 - val_accuracy: 0.7453 - val_loss: 0.5542
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8272 - loss: 0.3849 - val_accuracy: 0.7561 - val_loss: 0.5243
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8344 - loss: 0.3665 - val_accuracy: 0.7534 - val_loss: 0.5292
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8343 - loss: 0.3570 - val_accuracy: 0.7561 - val_loss

In [20]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=2)

scores

# In results, first is loss, second is accuracy

12/12 - 0s - 7ms/step - accuracy: 0.7534 - loss: 0.9874


[0.9874312281608582, 0.7533875107765198]

In [21]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.99
compile_metrics: 75.34%


# Simple RNN with one layer

In [22]:
n_steps = 36
n_inputs = 1


model = keras.models.Sequential([
    
    keras.layers.SimpleRNN(32, input_shape=[n_steps, n_inputs]), # 32 is no of neurons just select multiples of 16 can even be greater than 
    keras.layers.Dense(1, activation='sigmoid')
])

C:\Users\mural\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [23]:
from tensorflow.keras.callbacks import EarlyStopping


earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

callback = [earlystop]

In [24]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

# If multiclass, use "sparse_categorical_crossentropy" as the loss function
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])


history = model.fit(train_x, train_y, epochs=50,
                    validation_data=(test_x, test_y), callbacks=callback)

Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.6877 - loss: 0.5741 - val_accuracy: 0.7778 - val_loss: 0.4407
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8251 - loss: 0.3746 - val_accuracy: 0.7751 - val_loss: 0.4413
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8295 - loss: 0.3682 - val_accuracy: 0.7751 - val_loss: 0.4426
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8409 - loss: 0.3608 - val_accuracy: 0.7642 - val_loss: 0.4412
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8394 - loss: 0.3564 - val_accuracy: 0.7615 - val_loss: 0.4406
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8371 - loss: 0.3501 - val_accuracy: 0.7669 - val_loss: 0.4469
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8452 - loss: 0.3472 - val_accuracy: 0.7615 - val_loss: 0.4541
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8514 - loss: 0.3441 - val_accuracy: 0.7642 - v

In [25]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.4623696804046631, 0.7479674816131592]

In [26]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.46
compile_metrics: 74.80%


In [27]:
# Predictions are probabilities.

predictions = model.predict(test_x)

12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step


In [28]:
# Rounding the probabilities determines 1 or 0

np.round(predictions)

array([[1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],

In [29]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_y, np.round(predictions))

array([[128,  43],
       [ 50, 148]], dtype=int64)

# Deep RNN

**Be careful: when stacking RNN layers, you have to set "return_sequences" to True. This enables the layer to send a "sequence" of values to the next layer -- jut like how it uses a sequence of values for training.**

**Since the last layer is DENSE, it can't take sequence data. Therefore, you CANNOT return sequences from the previous layer. So, remove** `return_sequences` **from previous layer.**

In [30]:
n_steps = 36
n_inputs = 1


model = keras.models.Sequential([
    keras.layers.SimpleRNN(32, return_sequences=True, input_shape=[n_steps, n_inputs] ),
    keras.layers.SimpleRNN(32, return_sequences=True),
    keras.layers.SimpleRNN(32), 
    keras.layers.Dense(1, activation='sigmoid')
])


C:\Users\mural\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [31]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 9s 66ms/step - accuracy: 0.7868 - loss: 0.4778 - val_accuracy: 0.7696 - val_loss: 0.4792
Epoch 2/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.8102 - loss: 0.4091 - val_accuracy: 0.7751 - val_loss: 0.4645
Epoch 3/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.8303 - loss: 0.3918 - val_accuracy: 0.7940 - val_loss: 0.4542
Epoch 4/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.8331 - loss: 0.3701 - val_accuracy: 0.7724 - val_loss: 0.4497
Epoch 5/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.8328 - loss: 0.3703 - val_accuracy: 0.7778 - val_loss: 0.4486
Epoch 5: early stopping


In [32]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.4486066997051239, 0.7777777910232544]

In [33]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.45
compile_metrics: 77.78%


# LSTM with one layer

In [ ]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    
    keras.layers.LSTM(32, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

In [ ]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

In [ ]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


# LSTM with more layers

In [34]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.LSTM(32, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.LSTM(32, return_sequences=True),
    keras.layers.LSTM(32), #---> last layer return sequence must be false
    keras.layers.Dense(1, activation='sigmoid')
])


In [35]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 88ms/step - accuracy: 0.7063 - loss: 0.5459 - val_accuracy: 0.7805 - val_loss: 0.4660
Epoch 2/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.8139 - loss: 0.4067 - val_accuracy: 0.7805 - val_loss: 0.4548
Epoch 3/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.8221 - loss: 0.3935 - val_accuracy: 0.7859 - val_loss: 0.4476
Epoch 4/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - accuracy: 0.8332 - loss: 0.3900 - val_accuracy: 0.7751 - val_loss: 0.4474
Epoch 5/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - accuracy: 0.8312 - loss: 0.3814 - val_accuracy: 0.7751 - val_loss: 0.4460
Epoch 5: early stopping


In [36]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.44601932168006897, 0.7750677466392517]

In [37]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.45
compile_metrics: 77.51%


# GRU with one layer

In [38]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(32, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [39]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 6s 48ms/step - accuracy: 0.7082 - loss: 0.5212 - val_accuracy: 0.7859 - val_loss: 0.4466
Epoch 2/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.8263 - loss: 0.3853 - val_accuracy: 0.7832 - val_loss: 0.4388
Epoch 3/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.8304 - loss: 0.3768 - val_accuracy: 0.7859 - val_loss: 0.4368
Epoch 4/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.8309 - loss: 0.3730 - val_accuracy: 0.7859 - val_loss: 0.4347
Epoch 5/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8264 - loss: 0.3722 - val_accuracy: 0.7886 - val_loss: 0.4340
Epoch 6/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.8267 - loss: 0.3708 - val_accuracy: 0.7913 - val_loss: 0.4334
Epoch 7/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.8283 - loss: 0.3696 - val_accuracy: 0.7913 - val_loss: 0.4323
Epoch 8/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.8335 - loss: 0.3678 - val_accuracy: 0.7967 - v

In [40]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.46058234572410583, 0.7859078645706177]

In [41]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.46
compile_metrics: 78.59%


# GRU with more layers

In [42]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(32, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.GRU(32, return_sequences=True),
    keras.layers.GRU(32),
    keras.layers.Dense(1, activation='sigmoid')
])

In [43]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 14s 121ms/step - accuracy: 0.8249 - loss: 0.4534 - val_accuracy: 0.7886 - val_loss: 0.4711
Epoch 2/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - accuracy: 0.8284 - loss: 0.3892 - val_accuracy: 0.7669 - val_loss: 0.4433
Epoch 3/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - accuracy: 0.8348 - loss: 0.3828 - val_accuracy: 0.7724 - val_loss: 0.4473
Epoch 4/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - accuracy: 0.8406 - loss: 0.3842 - val_accuracy: 0.7696 - val_loss: 0.4442
Epoch 5/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - accuracy: 0.8389 - loss: 0.3815 - val_accuracy: 0.7778 - val_loss: 0.4451
Epoch 5: early stopping


In [44]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.44505032896995544, 0.7777777910232544]

In [45]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.45
compile_metrics: 77.78%


# Conv1D

In [46]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.Conv1D(filters=10, kernel_size=3, strides=2, padding="valid", input_shape=[n_steps, n_inputs]),
    keras.layers.Conv1D(filters=20, kernel_size=3, strides=1, padding="valid", dilation_rate=2),
    keras.layers.LSTM(32, return_sequences=True),
    keras.layers.LSTM(32),
    keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

C:\Users\mural\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 17, 10)         │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 13, 20)         │           620 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 13, 32)         │         6,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 32)             │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,797 (61.71 KB)

 Trainable params: 15,797 (61.71 KB)

 Non-trainable params: 0 (0.00 B)

In [47]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 9s 60ms/step - accuracy: 0.7832 - loss: 0.5211 - val_accuracy: 0.7669 - val_loss: 0.4843
Epoch 2/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.8076 - loss: 0.4159 - val_accuracy: 0.7669 - val_loss: 0.4800
Epoch 3/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.8119 - loss: 0.3981 - val_accuracy: 0.7480 - val_loss: 0.4966
Epoch 4/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.8269 - loss: 0.3921 - val_accuracy: 0.7588 - val_loss: 0.4815
Epoch 5/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.8190 - loss: 0.3841 - val_accuracy: 0.7588 - val_loss: 0.4941
Epoch 5: early stopping


In [48]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.4940841495990753, 0.7588075995445251]

In [49]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.49
compile_metrics: 75.88%
